In [ ]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# 필요한 패키지와 모듈을 불러옴
import numpy as np
import dlib
import cv2
from google.colab.patches import cv2_imshow
import os

In [ ]:
# eyes landmark 를 정의합니다.
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

In [ ]:
dataset_paths = ['gdrive/My Drive/CV/Face Landmark/image/Jo-front/', 'gdrive/My Drive/CV/Face Landmark/image/Mido-front/', 'gdrive/My Drive/CV/Face Landmark/image/unknown-front/']  # 원본 이미지 경로
output_paths = ['gdrive/My Drive/CV/Face Landmark/image/Jo-align/', 'gdrive/My Drive/CV/Face Landmark/image/Mido-align/', 'gdrive/My Drive/CV/Face Landmark/image/unknown-align/']   # output 이미지 경로
image_type = '.jpg'    # 이미지 타입

In [ ]:
predictor_file = 'gdrive/My Drive/CV/Face Landmark/shape_predictor_68_face_landmarks.dat'   # 이미 학습된 dlib 모델을 불러옴.
OUTPUT_SIZE = (300, 300)   # output 이미지 해상도

In [ ]:
detector = dlib.get_frontal_face_detector()       # 얼굴을 감지하는 detector 정의
predictor = dlib.shape_predictor(predictor_file)  # facial landmark 을 찾아주는 predictor 정의

In [ ]:
# rect 의 x, y, w, h 를 구하는 함수 정의
def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())

# Crop 할 좌표를 구하는 함수 정의
def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)  

In [ ]:
for (i, dataset_path) in enumerate(dataset_paths):
    output_path = output_paths[i]
    total_images = os.listdir(dataset_path)  # 해당 디렉토리에 있는 파일 리스트를 가져 옵니다.
    
    for idx in range(len(total_images)):
        input_file = dataset_path + str(idx+1) + image_type

        image = cv2.imread(input_file)   # 원본 이미지를 읽어옴
        image_origin = image.copy()      # 원본 이미지 copy

        (image_height, image_width) = image.shape[:2]
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        rects = detector(gray, 1)   # detector 에 의해 얼굴 감지

        for (i, rect) in enumerate(rects):
            (x, y, w, h) = getFaceDimension(rect)  # rect 의 x, y, w, h 를 구해온다.
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # 이미지에 boxing

            # predictor 에 의해서 감지된 landmark point 를 x, y 좌표로 계산
            points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])   
            show_parts = points[EYES]   

            right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")    # 오른쪽 눈의 중앙값 계산
            left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")      # 왼쪽 눈의 중앙값 계산

            eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0]
            eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1]
            degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180   # arctan2 로 조정 각도 계산

            eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2))       # 눈 사이의 간격을 Euclidean Distance 로 계산(실제 간격)
            aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0]   # alignment 된 눈 사이의 간격(조정 거리)
            scale = aligned_eye_distance / eye_distance                           # 조정된 거리의 스케일

            eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0]) // 2,
                    (left_eye_center[0,1] + right_eye_center[0,1]) // 2)          # 두 눈의 중앙점 간의 중앙점을 다시 계산 
                    
            rotate = cv2.getRotationMatrix2D(eyes_center, degree, scale)          # 이미지의 센터, 각도(+:반시계, -:시계), 스케일(확대/축소)
            rotated = cv2.warpAffine(image_origin, rotate, (image_width, image_height),
                flags=cv2.INTER_CUBIC)                                            # 원본 이미지 회전

            (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)  # rect 에서 Crop 할 이미지의 좌표값을 구해온다.
            croped = rotated[startY:endY, startX:endX]    # 회전된 이미지에서 Crop 될 이미지를 구한다.
            output = cv2.resize(croped, OUTPUT_SIZE)     # Crop 된 이미지를 Output 해상도에 맞게 resizing
                        
            output_file = output_path + str(idx+1) + image_type  # output 파일명
            cv2.imwrite(output_file, output)                     # output 이미지 저장
            print(output_file + " is saved")

gdrive/My Drive/CV/Face Landmark/image/Jo-align/1.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/2.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/3.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/4.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/5.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/6.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/7.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/8.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/9.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/10.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/11.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/12.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/13.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/14.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/15.jpg is saved
gdrive/My Drive/CV/Face Landmark/image/Jo-align/1